In [5]:
#caer canaro hali hazırda yüklü olmadıgından manuel yüklemek gerekir.
!pip install caer canaro



^C


In [6]:
#Gerekli kütüphaneler çağrılır.
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

ModuleNotFoundError: No module named 'caer'

In [ ]:
#resimler 80,80 olarak yeniden boyutlanır
IMG_SIZE = (80,80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
# karakter sözlüğü oluşturulur.
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
#caer.sort_dict() işlevi, bir sözlüğü belirli bir anahtar veya değere göre sıralamak için kullanılır. 
#descending=True parametresi, sıralamanın azalan sırada yapılmasını sağlar.
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [ ]:
#  En çok görsel içeren ilk 10 kategoriyi alır bunun için bir for döngüsü oluşturulur.
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
characters

In [ ]:
#eğitim verileri oluşturulur.caer.preprocess_from_dir fonksiyonu, bir dizinden görüntü verilerini önişlemek için kullanılır
#Bu fonksiyon, verileri yükleme, yeniden boyutlandırma, normalleştirme ve etiketleme gibi işlemleri otomatik olarak gerçekleştirir
#parametreleri;char_path: Görüntülerin yüklendiği dizin veya klasörün yolu.
#characters: Etiketlerin belirtildiği bir dizi veya liste. Her etiket, dizindeki bir alt klasörün adına karşılık gelir.
#channels (opsiyonel): Görüntülerin kanal sayısı. Öntanımlı değer 1'dir (gri tonlamalı). Renkli görüntüler için 3 kullanılabilir (RGB).
#IMG_SIZE (opsiyonel): Görüntülerin yeniden boyutlandırılacağı hedef boyut. Öntanımlı değer (80, 80) olarak belirlenmiştir.
#isShuffle (opsiyonel): Veri kümesinin karıştırılıp karıştırılmayacağını belirten bir bayrak. Öntanımlı değeri Truedur. Eğer True olarak ayarlanırsa, veri kümesi karıştırılır.
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
# örnek sayısı
len(train)

In [ ]:
# verileri matplotlib ile görselleştirme
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30)) #parametre;figsize: Oluşturulan figürün boyutunu belirler. Bu parametre, (genişlik, yükseklik) olarak belirtilir.
#parametreler; train[0][0]: Görselleştirilecek olan görüntü. cmap='gray': Renk haritası (colormap) belirten bir parametre
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
# Diziyi ve karşılık gelen etiketleri ayırma
#caer.sep_train fonku, eğitim veri kümesini eğitim ve doğrulama setlerine ayırmak için kullanılır
#train: Eğitim veri kümesi. Bu parametre, eğitimde kullanılacak olan görüntü veri kümesini temsil eder.
#IMG_SIZE=IMG_SIZE: Görüntülerin yeniden boyutlandırılacağı hedef boyut. 
#IMG_SIZE parametresi, tüm görüntülerin belirli bir boyuta (genişlik ve yükseklik) yeniden boyutlandırılmasını sağlar
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical

# Özellik setini normalleştirme ==> (0,1)
featureSet = caer.normalize(featureSet)
# Sayısal etiketleri ikili sınıf vektörlerine dönüştürme
labels = to_categorical(labels, len(characters))

In [ ]:
#caer.train_test_split fonku, veri kümesini eğitim ve doğrulama setlerine ayırmak için kullanılır. 
#featureSet: Girdi özellik vektörlerinin (X) olduğu veri kümesi
#labels: Etiketlerin (y) olduğu veri kümesi. Bu parametre, etiketleri içeren bir dizi veya liste olarak temsil edilir.
#val_ratio=.2: Doğrulama setinin oranı. Bu parametre, eğitim setinin yüzde kaçının doğrulama seti olarak kullanılacağını belirler.
x_train, x_val, y_train, y_val = caer.train_test_split(featureSet, labels, val_ratio=.2)

In [ ]:
del train
del featureSet
del labels 
gc.collect()

In [ ]:
# Kullanılabilir değişkenler
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# Görüntü veri oluşturucu
#canaro.generators.imageDataGenerator() fonksiyonu, veri artırma işlemlerini gerçekleştirmek için kullanılır
datagen = canaro.generators.imageDataGenerator()
#datagen.flow() fonksiyonu, veri artırma (data augmentation) işlemlerini uygulamak için bir veri jeneratörü oluşturur
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
# Model Ouşturma
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters), 
                                         loss='binary_crossentropy', decay=1e-7, learning_rate=0.001, momentum=0.9,
                                         nesterov=True)

In [ ]:
model.summary()

In [ ]:
# Modeli oluşturma

from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [ ]:
characters

# Testing
******

In [ ]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'

img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [ ]:
def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, IMG_SIZE)
    image = caer.reshape(image, IMG_SIZE, 1)
    return image

In [ ]:
#modelin tahmin yapması
predictions = model.predict(prepare(img))

In [ ]:
#aşağıdaki satır, modelin tahmin ettiği ilk çıktıyı kullanarak sınıflandırma sonucunu ekrana yazdırır.

print(characters[np.argmax(predictions[0])])